In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator, GlobalResults
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2'

#lstm
train_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])
test_lstm_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

#arimax
train_arimax_pred_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'arimax_lstm/{variant_co2}/pca_arimax_pred_train.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
test_arimax_pred_df = pd.read_csv(os.path.join(config.output_cleaned_hybrid, f'arimax_lstm/{variant_co2}/pca_arimax_pred_test.csv')).set_index(["year", "country"]).rename_axis(index={"country": config.additional_index})
train_arimax_pred_preprocessed, test_arimax_pred_preprocessed = data_preprocessor_lstm.preprocess_data(train_arimax_pred_df, test_arimax_pred_df)

In [3]:
train_lstm_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,oil_co2_t-2,...,population_t-1,gdp_t-1,temperature_change_from_co2_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,34,0.016985,0.010509,0.000000,0.000000,0.005568,0.000000,0.000000,0.449621,0.000000,...,0.017393,0.010347,0.006536,0.000000,0.005462,0.000000,0.000000,0.351582,0.000000,0.142663
1854,Stany Zjednoczone,34,0.017544,0.011437,0.006623,0.000000,0.006055,0.000000,0.000000,0.473915,0.000000,...,0.017965,0.011483,0.006536,0.000000,0.006149,0.000000,0.000000,0.355254,0.000000,0.148196
1855,Stany Zjednoczone,34,0.018120,0.012690,0.006623,0.000000,0.006817,0.000000,0.000000,0.478863,0.000000,...,0.018554,0.011840,0.013072,0.000000,0.006760,0.000000,0.000000,0.368256,0.000000,0.150416
1856,Stany Zjednoczone,34,0.018712,0.013083,0.013245,0.000000,0.007495,0.000000,0.000000,0.496390,0.000000,...,0.019160,0.011940,0.013072,0.000000,0.007779,0.000000,0.000000,0.372171,0.000000,0.154770
1857,Stany Zjednoczone,34,0.019322,0.013194,0.013245,0.000000,0.008625,0.000000,0.000000,0.501668,0.000000,...,0.019785,0.012561,0.013072,0.000000,0.008162,0.000000,0.000000,0.382568,0.000000,0.156695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,35,0.005112,0.027868,0.013245,0.002694,0.002540,0.003924,0.001725,0.046782,0.017724,...,0.005071,0.026320,0.013072,0.002757,0.002288,0.004482,0.001754,0.036047,0.017440,0.021023
1997,Szwecja,35,0.005147,0.029056,0.013245,0.003148,0.002537,0.004482,0.001754,0.048590,0.017440,...,0.005082,0.026822,0.013072,0.002606,0.002525,0.004283,0.001892,0.035421,0.018473,0.020478
1998,Szwecja,35,0.005158,0.029609,0.013245,0.002976,0.002800,0.004283,0.001892,0.047746,0.018473,...,0.005086,0.027630,0.013072,0.002351,0.002198,0.004762,0.001918,0.035754,0.017144,0.020634


In [4]:
train_arimax_pred_preprocessed

,,arimax_pred
year,country_index,
1853,Stany Zjednoczone,0.166322
1854,Stany Zjednoczone,0.224336
1855,Stany Zjednoczone,0.226217
1856,Stany Zjednoczone,0.227094
1857,Stany Zjednoczone,0.228886
...,...,...
1996,Szwecja,0.176708
1997,Szwecja,0.176796
1998,Szwecja,0.176554


In [5]:
#combine train df
train_hybrid_df = pd.merge(train_arimax_pred_preprocessed, train_lstm_df, left_index=True, right_index=True)
train_hybrid_df

,,arimax_pred,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,population_t-1,gdp_t-1,temperature_change_from_co2_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,0.166322,34,0.016985,0.010509,0.000000,0.000000,0.005568,0.000000,0.000000,0.449621,...,0.017393,0.010347,0.006536,0.000000,0.005462,0.000000,0.000000,0.351582,0.000000,0.142663
1854,Stany Zjednoczone,0.224336,34,0.017544,0.011437,0.006623,0.000000,0.006055,0.000000,0.000000,0.473915,...,0.017965,0.011483,0.006536,0.000000,0.006149,0.000000,0.000000,0.355254,0.000000,0.148196
1855,Stany Zjednoczone,0.226217,34,0.018120,0.012690,0.006623,0.000000,0.006817,0.000000,0.000000,0.478863,...,0.018554,0.011840,0.013072,0.000000,0.006760,0.000000,0.000000,0.368256,0.000000,0.150416
1856,Stany Zjednoczone,0.227094,34,0.018712,0.013083,0.013245,0.000000,0.007495,0.000000,0.000000,0.496390,...,0.019160,0.011940,0.013072,0.000000,0.007779,0.000000,0.000000,0.372171,0.000000,0.154770
1857,Stany Zjednoczone,0.228886,34,0.019322,0.013194,0.013245,0.000000,0.008625,0.000000,0.000000,0.501668,...,0.019785,0.012561,0.013072,0.000000,0.008162,0.000000,0.000000,0.382568,0.000000,0.156695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,0.176708,35,0.005112,0.027868,0.013245,0.002694,0.002540,0.003924,0.001725,0.046782,...,0.005071,0.026320,0.013072,0.002757,0.002288,0.004482,0.001754,0.036047,0.017440,0.021023
1997,Szwecja,0.176796,35,0.005147,0.029056,0.013245,0.003148,0.002537,0.004482,0.001754,0.048590,...,0.005082,0.026822,0.013072,0.002606,0.002525,0.004283,0.001892,0.035421,0.018473,0.020478
1998,Szwecja,0.176554,35,0.005158,0.029609,0.013245,0.002976,0.002800,0.004283,0.001892,0.047746,...,0.005086,0.027630,0.013072,0.002351,0.002198,0.004762,0.001918,0.035754,0.017144,0.020634


In [6]:
#combine test df
test_hybrid_df = pd.merge(test_arimax_pred_preprocessed, test_lstm_df, left_index=True, right_index=True)
test_hybrid_df

,,arimax_pred,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,population_t-1,gdp_t-1,temperature_change_from_co2_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1989,Stany Zjednoczone,0.454856,34,0.186605,0.980192,1.013245,0.080107,0.386410,0.088209,0.785603,0.170489,...,0.187127,0.925972,1.013072,0.070727,0.365490,0.101656,0.820841,0.131566,0.887696,0.744042
1990,Stany Zjednoczone,0.458081,34,0.188312,1.021423,1.026490,0.080768,0.405226,0.101656,0.820841,0.177344,...,0.188861,0.958024,1.026144,0.070761,0.369653,0.101017,0.874110,0.130467,0.887150,0.744836
1991,Stany Zjednoczone,0.454799,34,0.190056,1.056778,1.039735,0.080807,0.409841,0.101017,0.874110,0.175863,...,0.190911,0.974791,1.045752,0.071229,0.369909,0.616333,0.870813,0.136453,0.862972,0.729761
1992,Stany Zjednoczone,0.450702,34,0.192119,1.075274,1.059603,0.081341,0.410125,0.616333,0.870813,0.183932,...,0.193265,0.972084,1.058824,0.069638,0.366365,0.603152,0.886989,0.118116,0.840136,0.738090
1993,Stany Zjednoczone,0.452820,34,0.194488,1.072287,1.072848,0.079524,0.406197,0.603152,0.886989,0.159215,...,0.195648,1.006652,1.071895,0.070185,0.370468,0.598723,0.919292,0.101080,0.861291,0.749304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,Szwecja,0.177507,35,0.006089,0.052010,0.019868,0.003605,0.001879,0.010654,0.001542,0.053336,...,0.006096,0.048052,0.019608,0.003419,0.001650,0.010973,0.001605,0.039618,0.011526,0.019703
2020,Szwecja,0.177686,35,0.006179,0.053028,0.019868,0.003904,0.001829,0.010973,0.001605,0.053403,...,0.006175,0.049011,0.019608,0.002887,0.001654,0.007129,0.001515,0.039806,0.011298,0.019193
2021,Szwecja,0.177837,35,0.006258,0.054085,0.019868,0.003296,0.001834,0.007129,0.001515,0.053656,...,0.006232,0.047943,0.019608,0.002706,0.001355,0.005919,0.001227,0.039930,0.010443,0.019574


## Reshape the data

In [7]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_hybrid_df, test_hybrid_df)

## Build the model

In [8]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(96, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(80),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 96)             45312     
                                                                 
 dropout (Dropout)           (None, 1, 96)             0         
                                                                 
 lstm_1 (LSTM)               (None, 80)                56640     
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense (Dense)               (None, 1)                 81        
                                                                 
Total params: 102,033
Trainable params: 102,033
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [9]:
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size, callbacks=[early_stopping])

Epoch 1/100
99/99 [==============================] - 2s 5ms/step - loss: 0.0076
Epoch 2/100
99/99 [==============================] - 0s 3ms/step - loss: 0.0011
Epoch 3/100
99/99 [==============================] - 0s 4ms/step - loss: 6.7802e-04
Epoch 4/100
99/99 [==============================] - 0s 3ms/step - loss: 6.0598e-04
Epoch 5/100
99/99 [==============================] - 0s 2ms/step - loss: 6.2824e-04
Epoch 6/100
99/99 [==============================] - 0s 2ms/step - loss: 6.0418e-04
Epoch 7/100
99/99 [==============================] - 0s 3ms/step - loss: 4.5828e-04
Epoch 8/100
99/99 [==============================] - 0s 3ms/step - loss: 4.1529e-04
Epoch 9/100
99/99 [==============================] - 0s 3ms/step - loss: 4.8389e-04
Epoch 10/100
99/99 [==============================] - 0s 2ms/step - loss: 3.7965e-04
Epoch 11/100
99/99 [==============================] - 0s 2ms/step - loss: 4.1231e-04
Epoch 12/100
99/99 [==============================] - 0s 2ms/step - loss: 3.9601e-

In [10]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_pca_arimax_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [11]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_hybrid_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_hybrid_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_hybrid_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_hybrid_df.shape[1]-2)

26/26 [==============================] - 0s 519us/step


In [12]:
train_df_reset = train_hybrid_df.reset_index()
test_df_reset = test_hybrid_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_hybrid_arimax_lstm, f'arimax_lstm_{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_hybrid_arimax_lstm, f'arimax_lstm_{variant_co2}_test.csv'))

In [13]:
current_model = "arimax_pca_lstm"

global_csv_path = os.path.join(config.predictions, f'combined_results_{variant_co2}.csv')
global_results = GlobalResults(global_csv_path, keys=["country", "year", "set"])

train = train_results.copy()
test = test_results.copy()

if "year" not in train.columns:
    train = train.reset_index()
if "year" not in test.columns:
    test = test.reset_index()

train["set"] = "train"
test["set"] = "test"

train = train.rename(columns={
    "co2_predicted": f"co2_predicted_{current_model}",
    "co2_actual": f"co2_actual_{current_model}"
})
test = test.rename(columns={
    "co2_predicted": f"co2_predicted_{current_model}",
    "co2_actual": f"co2_actual_{current_model}"
})

new_results_df = pd.concat([train, test], axis=0)
new_results_df = new_results_df.sort_values(by=["year", "country", "set"])

global_results.append_results(new_results_df)

## Charts

In [14]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm)
charts.generate_line_plot_one_dataset(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm)
charts.generate_scatter_plot_one_dataset(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = f'{variant_co2}_pca', model_output_file=config.predictions_hybrid_arimax_lstm, dataset_type='test')

## Metrics

In [15]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = f'arimax_lstm_{variant_co2}_pca', model_output_file=config.metrics_hybrid)